In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms
from tqdm import tqdm
import os
from PIL import Image

In [44]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomImageFolder(datasets.ImageFolder):
    def __init__(self, root, transform=None, loader=None):
        super().__init__(root, transform=transform, loader=loader)

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)

        if sample is None:
            print(f"Skipping corrupted image: {path}")
            return self.__getitem__((index + 1) % len(self.samples))

        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

def safe_loader(path):
    try:
        img = Image.open(path)
        img.verify()  # Verify if the image is corrupted
        img = Image.open(path)  # Reopen after verification to ensure it's loaded properly
        return img.convert("RGB")
    except Exception as e:
        print(f"Skipping corrupted image: {path}")
        return None

dataset = CustomImageFolder(root='/kaggle/input/microsoft-catsvsdogs-dataset/PetImages', transform=transform, loader=safe_loader)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

val_size = int(0.1 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [45]:
model = models.resnet18(pretrained=True)
# model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [46]:
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 2)
)

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [49]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    correct_preds = 0
    total_samples = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels)
        total_samples += labels.size(0)

        running_loss += loss.item()

        if (batch_idx + 1) % 50 == 0:
            batch_accuracy = (correct_preds.double() / total_samples).item()
            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], "
                  f"Loss: {loss.item():.4f}, Accuracy: {batch_accuracy:.4f}")


    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_preds.double() / total_samples
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    model.eval()
    val_loss = 0.0
    val_correct_preds = 0
    val_total_samples = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_correct_preds += torch.sum(preds == labels)
            val_total_samples += labels.size(0)

    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = val_correct_preds.double() / val_total_samples
    print(f"Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_accuracy:.4f}")


Epoch [1/10], Batch [50/563], Loss: 0.1046, Accuracy: 0.8819
Epoch [1/10], Batch [100/563], Loss: 0.1603, Accuracy: 0.8978
Epoch [1/10], Batch [150/563], Loss: 0.1233, Accuracy: 0.9102
Epoch [1/10], Batch [200/563], Loss: 0.1237, Accuracy: 0.9142
Epoch [1/10], Batch [250/563], Loss: 0.1446, Accuracy: 0.9170
Epoch [1/10], Batch [300/563], Loss: 0.1484, Accuracy: 0.9206
Epoch [1/10], Batch [350/563], Loss: 0.1396, Accuracy: 0.9248
Epoch [1/10], Batch [400/563], Loss: 0.1400, Accuracy: 0.9270
Epoch [1/10], Batch [450/563], Loss: 0.0576, Accuracy: 0.9287
Epoch [1/10], Batch [500/563], Loss: 0.1100, Accuracy: 0.9300
Epoch [1/10], Batch [550/563], Loss: 0.0875, Accuracy: 0.9306
Epoch [1/10], Loss: 0.1724, Accuracy: 0.9308
Skipping corrupted image: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg
Skipping corrupted image: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg
Epoch [1/10], Validation Loss: 0.1099, Validation Accuracy: 0.9555
Epoch [2/10], Batch [

In [ ]:
model.eval()
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        correct_preds += torch.sum(preds == labels)
        total_samples += labels.size(0)

In [51]:
test_acc = correct_preds.double() / total_samples
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.9346
